In [ ]:
pip install openai-agents

In [23]:
import os
os.environ["OPENAI_API_KEY"] = "API KEY"

In [ ]:
from agents import Agent, Runner

# エージェントの作成
agent = Agent(name="Assistant", instructions="あなたは役に立つアシスタントです")

# エージェントの実行
result = Runner.run_sync(agent, "家族について俳句を書いてください。")
print(result.final_output)

In [ ]:
from agents import Agent, InputGuardrail, GuardrailFunctionOutput, Runner
from pydantic import BaseModel
import asyncio

# 数学家庭教師エージェントの定義
math_tutor_agent = Agent(
    name="数学家庭教師",
    handoff_description="数学の質問に特化したエージェント",
    instructions="あなたは数学の問題を手助けします。各ステップで論理的な説明を行い、例を含めてください。",
)

# 歴史家庭教師エージェントの定義
history_tutor_agent = Agent(
    name="歴史家庭教師",
    handoff_description="歴史の質問に特化したエージェント",
    instructions="あなたは歴史に関する質問を手助けします。重要な出来事や背景を明確に説明してください。",
)

# 宿題に関する質問かどうかを判断するモデル
class HomeworkOutput(BaseModel):
    is_homework: bool  # 宿題に関する質問かどうか
    reasoning: str  # その判断の理由

# ガードレールエージェントの定義
guardrail_agent = Agent(
    name="ガードレールチェック",
    instructions="ユーザーが宿題について質問しているか確認してください。",
    output_type=HomeworkOutput,
)

# ガードレール関数の定義
async def homework_guardrail(ctx, agent, input_data):
    result = await Runner.run(guardrail_agent, input_data, context=ctx.context)
    final_output = result.final_output_as(HomeworkOutput)
    return GuardrailFunctionOutput(
        output_info=final_output,
        tripwire_triggered=not final_output.is_homework,
    )

# トリアージエージェントの定義
triage_agent = Agent(
    name="トリアージエージェント",
    instructions="ユーザーの質問が宿題に関するものかを判断し、適切なエージェントを選択してください。",
    handoffs=[history_tutor_agent, math_tutor_agent], # ハンドオフ
    input_guardrails=[
        InputGuardrail(guardrail_function=homework_guardrail),
    ],
)

# メイン
async def main():
    # 「アメリカの最初の大統領は誰ですか？」の質問を処理
    try:
        user_question = input("質問を入力してください（例：アメリカの初代大統領は誰ですか？、2 + 2 はいくつですか？）：")
        result = await Runner.run(triage_agent,user_question)
        print(result.final_output)
    except Exception as e:
        print("エラー:私には役不足です。ごめんなさい。")

# メインの実行
if __name__ == "__main__":
    asyncio.run(main())

In [ ]:
from agents import Agent, Runner
import asyncio

# 数学家庭教師エージェントの定義
math_tutor_agent = Agent(
    name="数学家庭教師",
    handoff_description="数学の質問に特化したエージェント",
    instructions="あなたは数学の問題を手助けします。各ステップで論理的な説明を行い、例を含めてください。",
)

# 歴史家庭教師エージェントの定義
history_tutor_agent = Agent(
    name="歴史家庭教師",
    handoff_description="歴史の質問に特化したエージェント",
    instructions="あなたは歴史に関する質問を手助けします。重要な出来事や背景を明確に説明してください。",
)

# トリアージエージェントの定義
triage_agent = Agent(
    name="トリアージエージェント",
    instructions="ユーザーの質問の内容を判断し、数学または歴史の適切な家庭教師エージェントに転送してください。",
    handoffs=[history_tutor_agent, math_tutor_agent], # ハンドオフ
)

# メイン
async def main():
    # 標準入力から質問を受け取る
    user_question = input("質問を入力してください（例：アメリカの初代大統領は誰ですか？、2 + 2 はいくつですか？）：")
    result = await Runner.run(triage_agent, user_question)
    print(result.final_output)

# メインの実行
if __name__ == "__main__":
    asyncio.run(main())

In [ ]:
from agents import Agent, Runner
import asyncio
from typing import List

# 感情分析エージェント
sentiment_analyzer_agent = Agent(
    name="感情分析エージェント",
    instructions="""与えられた製品レビューの感情を分析し、ポジティブ、ネガティブ、またはニュートラルに分類してください。
レビューごとに感情と、その理由となる根拠を簡潔に示してください。""",
    model="gpt-4o",
)

# 機能評価エージェント
feature_evaluator_agent = Agent(
    name="機能評価エージェント",
    instructions="""与えられた製品レビューから言及されている主要な機能（例：バッテリー、カメラ、デザイン、操作性、ソフトウェアなど）を抽出し、それぞれの機能に対する評価（良い点、悪い点など）をレビューに基づいて特定してください。""",
    model="gpt-4o",
)

# 競合製品比較エージェント
competitor_comparison_agent = Agent(
    name="競合製品比較エージェント",
    instructions="""与えられた製品レビューの中に競合他社の製品に関する言及があれば、それを特定し、レビュー内で比較されている内容（価格、性能など）と、それに対する感情を分析してください。""",
    model="gpt-4o",
)

async def analyze_reviews(review):
    """複数のエージェントにレビューを並行して分析させる関数"""
    sentiment_tasks = await Runner.run(sentiment_analyzer_agent, review)
    feature_tasks = await Runner.run(feature_evaluator_agent, review)
    competitor_tasks = await Runner.run(competitor_comparison_agent, review)

    # 結果の統合と表示
    print("\n--- 感情分析 ---")
    print(sentiment_tasks.final_output)

    print("\n--- 機能評価 ---")
    print(feature_tasks.final_output)

    print("\n--- 競合製品比較 ---")
    print(competitor_tasks.final_output)

# メイン
async def main():
    # 製品レビューのリスト（例）
    product_reviews = input("製品レビュー：")
    await analyze_reviews(product_reviews)

if __name__ == "__main__":
    asyncio.run(main())